<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>This notebook will introduce you to the Allen Brain Observatory dataset and SDK functions. 

<p>We want to look at signal correlations between all the neurons in a given experiment and see whether this is related to the distance between the cells.
</div>

In [2]:
#make sure your drive path is correct!
drive_path = '/local1/Documents/projects/cam_analysis/boc/'


In [1]:
# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import sys


import matplotlib.pyplot as plt
%matplotlib inline

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>The main entry point is the `BrainObservatoryCache` class.  This class is responsible for downloading any requested data or metadata as needed and storing it in well known locations.  For this workshop, all of the data has been preloaded onto the hard drives you have received.

<p>We begin by importing the `BrainObservatoryCache` class and instantiating it.

<p>`manifest_path` is a path to the manifest file.  We will use the manifest file preloaded onto your Workshop hard drives.  Make sure that `drive_path` is set correctly for your platform.  (See the first cell in this notebook.)
</div>


In [3]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache

# manifest_file = os.path.join(drive_path,'manifest.json')
manifest_file="/data/dynamic-brain-workshop/brain_observatory_cache/brain_observatory_manifest.json"

boc = BrainObservatoryCache(manifest_file=manifest_file)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 1.1:**  Get information about what's in the dataset from BrainObservatoryCache

<p>The following methods for BrainObservatoryCache retrieve the available depths, cre lines, areas, and stimuli.  Notice that these parameters outline the 'data cube'.
</div>

In [4]:
# Download a list of all targeted areas
targeted_structures = boc.get_all_targeted_structures()
print 'all targeted structures: ' + str(targeted_structures)

# Download a list of all imaging depths
depths = boc.get_all_imaging_depths()
print 'all imaging depths: ' + str(depths)

# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print 'all cre lines: ' + str(cre_lines)

# Download a list of all stimuli
stims = boc.get_all_stimuli()
print 'all stimuli: ' + str(stims)

all targeted structures: [u'VISal', u'VISam', u'VISl', u'VISp', u'VISpm', u'VISrl']
all imaging depths: [175, 265, 275, 300, 320, 325, 335, 350, 365, 375, 435]
all cre lines: [u'Cux2-CreERT2', u'Emx1-IRES-Cre', u'Nr5a1-Cre', u'Rbp4-Cre_KL100', u'Rorb-IRES2-Cre', u'Scnn1a-Tg3-Cre']
all stimuli: ['drifting_gratings', 'locally_sparse_noise', 'locally_sparse_noise_4deg', 'locally_sparse_noise_8deg', 'natural_movie_one', 'natural_movie_three', 'natural_movie_two', 'natural_scenes', 'spontaneous', 'static_gratings']


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 1.2:**  Use tab completion in Jupyter to see what other methods the BrainObservatoryCache has.
</div>

In [ ]:
# Hit the 'tab' key with the cursor just after the '.'
boc.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Experiment containers</h2>
<p>The experiment container describes a set of 3 experiment sessions performed at the same location (targeted area and imaging depth) in the same mouse that targets the same set of cells. Each experiment container has a unique ID number.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 1.3:**  Choose a visual area and Cre line from the lists above to examine in the rest of the notebook
</div>

In [8]:
visual_area = 'VISp'
cre_line ='Cux2-CreERT2'

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 1.4:**  Get the list of all the experiment containers for that area and Cre line combination.
</div>

In [11]:
exps = boc.get_experiment_containers(targeted_structures=[visual_area], cre_lines=[cre_line])

In [12]:
#make a pandas DataFrame to see what information we have on these experiments
pd.DataFrame(exps)

,cre_line,donor_name,failed,id,imaging_depth,reporter_line,specimen_name,tags,targeted_structure
0,Nr5a1-Cre,248894,False,528792730,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-248894,[],VISp
1,Nr5a1-Cre,248895,False,528799602,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-248895,[],VISp
2,Nr5a1-Cre,250606,False,531100608,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-250606,[],VISp
3,Nr5a1-Cre,257786,False,538803515,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-257786,[],VISp
4,Nr5a1-Cre,250605,False,529763300,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-250605,[],VISp
5,Nr5a1-Cre,255445,False,532233174,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-255445,[],VISp
6,Nr5a1-Cre,250606,False,539291370,335,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-250606,[],VISp
7,Nr5a1-Cre,279430,False,569811199,300,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-279430,[],VISp
8,Nr5a1-Cre,286360,False,570278595,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-286360,[],VISp


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 1.5:**  Pick a random experiment from this selection
</div>

In [13]:
expt_container_id = np.random.choice(exps)['id']
print expt_container_id

539291370


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 1.6:** Get information about all of the experiment <strong>sessions</strong> in your experiment <strong>container</strong>.  This is accomplished with the `get_ophys_experiments` method.  
</div>

In [16]:
expt_session_info = boc.get_ophys_experiments(experiment_container_ids=[expt_container_id])
print(expt_session_info)
print len(expt_session_info)

[{'imaging_depth': 335, 'experiment_container_id': 539291370, 'reporter_line': u'Ai93(TITL-GCaMP6f)', 'targeted_structure': u'VISp', 'cre_line': u'Nr5a1-Cre', 'session_type': u'three_session_B', 'donor_name': u'250606', 'id': 539291372, 'acquisition_age_days': 119, 'specimen_name': u'Nr5a1-Cre;Camk2a-tTA;Ai93-250606'}, {'imaging_depth': 335, 'experiment_container_id': 539291370, 'reporter_line': u'Ai93(TITL-GCaMP6f)', 'targeted_structure': u'VISp', 'cre_line': u'Nr5a1-Cre', 'session_type': u'three_session_C', 'donor_name': u'250606', 'id': 539540432, 'acquisition_age_days': 120, 'specimen_name': u'Nr5a1-Cre;Camk2a-tTA;Ai93-250606'}, {'imaging_depth': 335, 'experiment_container_id': 539291370, 'reporter_line': u'Ai93(TITL-GCaMP6f)', 'targeted_structure': u'VISp', 'cre_line': u'Nr5a1-Cre', 'session_type': u'three_session_A', 'donor_name': u'250606', 'id': 539487468, 'acquisition_age_days': 120, 'specimen_name': u'Nr5a1-Cre;Camk2a-tTA;Ai93-250606'}]
3


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>`get_experiment_containers` returns a list of dictionaries that contain information about <b>experiment containers</b>.

<p>`get_ophys_experiments` returns a list of dictionaries that contain information about <b>experiment sessions</b>.  Here we are using keyword arguments to return just those experiment sessions that belong to our experiment container. We could have used targeted structures and Cre lines to find all the experiment sessions in our chosen location.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 1.7:** Create a dataframe of the experiment sessions in your chosen container.  
</div>

In [18]:
expt_session_info_df = pd.DataFrame(expt_session_info)
expt_session_info_df

,acquisition_age_days,cre_line,donor_name,experiment_container_id,id,imaging_depth,reporter_line,session_type,specimen_name,targeted_structure
0,119,Nr5a1-Cre,250606,539291370,539291372,335,Ai93(TITL-GCaMP6f),three_session_B,Nr5a1-Cre;Camk2a-tTA;Ai93-250606,VISp
1,120,Nr5a1-Cre,250606,539291370,539540432,335,Ai93(TITL-GCaMP6f),three_session_C,Nr5a1-Cre;Camk2a-tTA;Ai93-250606,VISp
2,120,Nr5a1-Cre,250606,539291370,539487468,335,Ai93(TITL-GCaMP6f),three_session_A,Nr5a1-Cre;Camk2a-tTA;Ai93-250606,VISp


How many sessions are in your experiment container? What is different about each session? 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 1.3:**  Find the session id for the session that has Natural Scenes as a stimulus for your chosen experiment container.  Save this as `session_id`. (Hint: you can either know which session type contains the Natural Scenes stimulus, or you can use the stimulus to select the experiment session)
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>We will be using the session you have recorded in `session_id` for much of the remainder of this notebook.

<h2>The Dataset Object</h2>
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 2.1:**  Create a data_set object for this experiment session.

The data_set object contains methods and info for a single experiment session (one of the 3 in the experiment container)
</div>

In [21]:
# make a data_set object
data_set = boc.get_ophys_experiment_data(ophys_experiment_id=session_id)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 2.2:** Use either `dir` or tab-completion to find out what methods the new `data_set` object has.
</div>

In [ ]:
data_set.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>Using the methods you find, perform the following exercises.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.1:** Get the metadata for your data set. How old was the mouse in this experiment?  Was it male or female?  
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.3:** Get the max projection image for your data set. Display it.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.5:** Get the roi mask for all the cells.  (Hint:  There are two methods that return roi masks.  In one of them masks are returned as lists of python objects.  What methods do they have?  What is the type of this object?)  What is the size and shape of the mask? How many cells are in this experiment session?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.6:** Pick one cell (at random) and plot the mask overlayed on the max projection.  (Hint:  imshow has an optional parameter called `alpha`.) What is the structure of the mask - how can you find the position of the mask?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.7:** Calculate the centroid of the ROI mask. Verify that you've calculated this properly by plotting it on top of the mask
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.7:** Find the centroid of each roi mask and calculate the distance between all the ROIs
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 2.8:** Plot the distribution of distances. Only count each distance once! (Hint: numpy has functions to select the upper or lower triangle of an array. Look for triu, or triu_indices. Be sure to eliminate the diagonal itself)
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>The Analysis Object</h2>


<p>The analysis objects summarize the trial data for a stimulus type and provide convenient DataFrame objects.  
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Task 3.1:**  Import the `NaturalScenes` object and instatiate it with `data_set`.
</div>

In [38]:
from allensdk.brain_observatory.natural_scenes import NaturalScenes
NS = NaturalScenes(data_set)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.1:** Get the "signal correlations" of the responses to natural scenes. You can opt for pearson or spearman - pearson is faster but spearman is better. (Either this way will take a bit of time to run  - this is a great time for a bathroom break) What is the shape of this object?  The methods will return a tuple of length two.  The first value is the signal correlation, the second the p values for .
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.2:** Plot the signal correlation vs distance for all pairs of cells.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.3:** Find the most signal-correlated pair of neurons in this experiment and plot their ROIs.
</div>

We want to look at the trial-by-trials responses of these cells to their preferred stimuli. To do this we're going to pull out the responses from our analysis object.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>sweep_response is a DataFrame that contains the dF/F response of each cell during each stimulus trial. It shares its index with stim_table. Each cell contains a timeseries that extends from 1 second prior to the start of the trial to 1 second after the end of the trial. The sweep_response table is organized as cells (columns) for each sweep (rows)

<p>mean_sweep_response provides the time-averaged dF/F for each trial.

<p>The stimulus_table object contains information about the stimulus. Use data_set.get_stimulus_table('natural_scenes') to get this object. What does it contain?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.4:** Get the mean sweep responses of your two cells to natural images. What shape are these?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.5:** Get the natural scenes stimulus table. What shape is it? What are its columns?
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.6:** Use the stimulus table to get the mean sweep responses to natural image 10. Plot your two cells' mean sweep responses to it.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.7:** Get the trial-averaged mean sweep responses to each natural scene for your two cells. (Hint: in addition to using mean_sweep_response and the stimulus table, try checking NS.response.)
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.8:** Plot the neurons' preferred images. (Hint: get the stimulus_template object from the data_set. What shape is it?) (Hint2: pay attention to the image indices and the frame labels in the stimulus table)
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 3.9:** Plot the two neurons' trial-by-trial sweep responses to their preferred images. 
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Mouse Behavior</h2>


<p>The data_set and analysis objects also contain information about the mouse's running speed and eye tracking. (Not all experiments have eye tracking information.) The running speed, eye position and pupil size can be extracted from the data_set object using appropriately named functions. The running speed has been analyzed the same way as each fluorescent trace, and contained in the final column of the sweep_response and mean_sweep_response dataframes named "dx". 
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 4.1:** Plot the mouse's running speed over the whole experiment and on the same trials you looked at responses of above.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 4.2:** Check if your experiment has information about the mouse's pupil diameter. If so, plot it on the sweeps of each neuron's preferred image.
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>The Cell Specimens Object</h2>


<p>The cell specimens object contains summary information about all cells in the brain observatory dataset. Let's find your two cells in it and compare them! First, load the cell_specimens object and see what types of information it contains about each cell.
</div>

In [ ]:
cell_specimens = boc.get_cell_specimens()
print cell_specimens.keys()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 5.1:** Find your two cells' cell specimen ids in the data_set object.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p>**Exercise 5.2:** Get the cell_specimens information for just your two cells. (Hint: when you create the cell_specimens object, you can filter by cell specimen id.) What is each of their response reliability for natural scenes? How does it compare to their reliability for static gratings?
</div>